In [1]:
import os
from cralds_base.dataio import read_file
from cralds.dataio.fileio.read.specific.trios_rheo_txt import TriosRheoReader
import sys
sys.path.append(r'../src/')
from hermes_rheo.transforms.rheo_analysis import RheoAnalysis 
from hermes_rheo.transforms.automated_mastercurve import AutomatedMasterCurve
from hermes_rheo.transforms.states_to_conditions import StatesToConditions

import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")

In [2]:
filepath = r'Q:\Projects\Rheology_OP\CRAL_hermes\damien_vadillo\SoR-2024\UV\032-Hermes - 02244-1 chirp phi 90- 2024.05.22.txt' # Chirp data
experiment = read_file(filepath, create_composite_datasets=True)
pipeline = RheoAnalysis() + StatesToConditions(state='time', method='first point')

processed_experiment = pipeline(experiment)
print(processed_experiment)

ExperimentSet
-------------

Summary
-------
Number of Experiments: 100

Varied Conditions
-----------------
method, time

index	n_rep	method	time	Data Types	Data Types	Data Types	
0	1	Step name	Arbitrary Wave - 1	0.00216635642573237	storage modulus(Pa)=f(angular frequency[rad/s]) (39 points)	time(s)=f(temperature[°C]) (2499 points)	strain filtered masked(a.u)=f(time masked[s]) (1400 points)	
1	1	Step name	Arbitrary Wave - 2	10.6364994049072	storage modulus(Pa)=f(angular frequency[rad/s]) (39 points)	time(s)=f(temperature[°C]) (2499 points)	strain filtered masked(a.u)=f(time masked[s]) (1400 points)	
2	1	Step name	Arbitrary Wave - 3	21.2708339691162	storage modulus(Pa)=f(angular frequency[rad/s]) (39 points)	time(s)=f(temperature[°C]) (2499 points)	strain filtered masked(a.u)=f(time masked[s]) (1400 points)	
3	1	Step name	Arbitrary Wave - 4	31.9051685333252	storage modulus(Pa)=f(angular frequency[rad/s]) (39 points)	time(s)=f(temperature[°C]) (2499 points)	strain filtered masked(a.u)=f

In [3]:
import numpy as np
import copy

# Initialize lists to store the extracted data
angular_frequencies = []
storage_moduli = []
loss_moduli = []
complex_moduli = []
complex_viscosities = []
phase_angles = []
step_times = []
strains = []
stresses = []
temperatures = []
time_states = []
# Number of steps 
steps = len(processed_experiment)

# Process each experiment to extract data
for step in range(steps):
    dataset_frequency = processed_experiment[step][0].datasets[0]
    time_state = processed_experiment[step][0].conditions['time']
    time_states.append(time_state)

    angular_frequency = copy.deepcopy(dataset_frequency.x_values)
    storage_modulus = copy.deepcopy(dataset_frequency.y_values)

    dataset_frequency.switch_coordinates('loss modulus', 'complex modulus')
    loss_modulus = copy.deepcopy(dataset_frequency.x_values)
    complex_modulus = copy.deepcopy(dataset_frequency.y_values)

    dataset_frequency.switch_coordinates('complex viscosity', 'tan(delta)')
    complex_viscosity = copy.deepcopy(dataset_frequency.x_values)
    phase_angle = copy.deepcopy(dataset_frequency.y_values)

    angular_frequencies.append(angular_frequency)
    storage_moduli.append(storage_modulus)
    loss_moduli.append(loss_modulus)
    complex_moduli.append(complex_modulus)
    complex_viscosities.append(complex_viscosity)
    phase_angles.append(phase_angle)

    dataset_time = processed_experiment[step][0].datasets[1]
    dataset_time.switch_coordinates('time_cut', 'strain filtered')
    step_time = copy.deepcopy(dataset_time.x_values)
    strain = copy.deepcopy(dataset_time.y_values)

    dataset_time.switch_coordinates('temperature', 'stress filtered')
    temperature = copy.deepcopy(dataset_time.x_values)
    stress = copy.deepcopy(dataset_time.y_values)

    step_times.append(step_time)
    strains.append(strain)
    temperatures.append(temperature)
    stresses.append(stress)


In [8]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Define the target frequency for moduli selection
target_frequency = 6.28  # Provide the frequency in rad/s

# Define colors for consistency across the plots
strain_color = 'green'
stress_color = 'orange'
modulus_color_Gprime = 'red'
modulus_color_Gdoubleprime = 'blue'

# Initialize lists to accumulate G' and G'' values over the steps
accumulated_Gprime_values = []
accumulated_Gdoubleprime_values = []
accumulated_time_states = []

# Create the folder to save PNGs if it doesn't exist
output_dir = "032-Hermes - 02244-1 chirp phi 90- 2024.05.22"
os.makedirs(output_dir, exist_ok=True)

# Function to find the index of the closest frequency to the target
def find_closest_index(frequencies, target_frequency):
    return np.abs(np.array(frequencies) - target_frequency).argmin()

# Loop over each step and generate figures, but do not display them
for step in range(steps):
    # Create a figure and define the 2x2 grid layout
    fig, axs = plt.subplots(2, 2, figsize=(9, 7))

    # Set a uniform font size for all labels and adjust tick label size
    label_fontsize = 16
    tick_labelsize = 14
    marker_size = 8  # Increased marker size for plots 3 and 4

    # Top-left: Strain vs Time (using Greek gamma for strain, green color)
    axs[0, 0].plot(step_times[step], strains[step], color=strain_color)
    axs[0, 0].set_xlabel(r'Wave time (s)', fontsize=label_fontsize)
    axs[0, 0].set_ylabel(r'$\gamma$', fontsize=label_fontsize)
    axs[0, 0].set_xlim(0, 5)
    axs[0, 0].tick_params(labelsize=tick_labelsize)

    # Top-right: Stress vs Time (using Greek gamma, orange color)
    axs[0, 1].plot(step_times[step], stresses[step], color=stress_color)
    axs[0, 1].set_xlabel(r'Wave time (s)', fontsize=label_fontsize)
    axs[0, 1].set_ylabel('$\sigma$ (Pa)', fontsize=label_fontsize)
    axs[0, 1].set_xlim(0, 5)
    axs[0, 1].tick_params(labelsize=tick_labelsize)

    # Bottom-left: Storage and Loss Modulus vs Frequency (log-log scale, same color/markers as bottom-right)
    axs[1, 0].plot(angular_frequencies[step], storage_moduli[step], color=modulus_color_Gprime, linestyle='-', marker='s', markersize=marker_size)
    axs[1, 0].plot(angular_frequencies[step], loss_moduli[step], color=modulus_color_Gdoubleprime, linestyle='--', marker='^', markersize=marker_size)
    axs[1, 0].set_xlabel(r'$\omega$ (rad/s)', fontsize=label_fontsize)
    axs[1, 0].set_ylabel(r"$G'$, $G''$ (Pa)", fontsize=label_fontsize)  # Same y-label as bottom-right
    axs[1, 0].set_xscale('log')  # Log-log scale
    axs[1, 0].set_yscale('log')
    axs[1, 0].set_ylim(1e-1, 1e7)
    axs[1, 0].tick_params(labelsize=tick_labelsize)

    # Bottom-right: Moduli (G', G'') vs Time States
    # Find the index corresponding to the target frequency
    frequency_index = find_closest_index(angular_frequencies[step], target_frequency)

    # Get the corresponding G' and G'' values for that frequency
    current_Gprime_value = storage_moduli[step][frequency_index]
    current_Gdoubleprime_value = loss_moduli[step][frequency_index]

    # Accumulate the values to plot them over all steps
    accumulated_Gprime_values.append(current_Gprime_value)
    accumulated_Gdoubleprime_values.append(current_Gdoubleprime_value)
    accumulated_time_states.append(time_states[step])

    axs[1, 1].plot(accumulated_time_states, accumulated_Gprime_values, marker='s', label=r"$G'$", color=modulus_color_Gprime, markersize=marker_size, linestyle='-')
    axs[1, 1].plot(accumulated_time_states, accumulated_Gdoubleprime_values, marker='^', label=r"$G''$", color=modulus_color_Gdoubleprime, markersize=marker_size, markerfacecolor='none', linestyle='--')
    
    axs[1, 1].set_xlabel('Experiment Time (s)', fontsize=label_fontsize)
    axs[1, 1].set_ylabel(r"$G'$, $G''$ (Pa)", fontsize=label_fontsize)
    axs[1, 1].set_yscale('log')  # Log scale for y-axis
    axs[1, 1].set_ylim(1e-1, 1e7)  # Set limits for y-axis
    axs[1, 1].set_xlim(0, 1150)  # Set limits for x-axis
    axs[1, 1].tick_params(labelsize=tick_labelsize)
    axs[1, 1].legend(loc=4, frameon=False, prop={'size': 12})

    # Save the figure as a PNG file in the "movie_tcs" directory
    output_filename = os.path.join(output_dir, f"plot_{step}.png")
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300)
    plt.close(fig)

In [9]:
# Now create a video from the saved PNG plots
image_files = sorted([img for img in os.listdir(output_dir) if img.endswith(".png")], key=lambda x: int(x.split('_')[1].split('.')[0]))

# Define the codec and create a VideoWriter object
frame = cv2.imread(os.path.join(output_dir, image_files[0]))
height, width, layers = frame.shape
video_filename = '032-Hermes - 02244-1 chirp phi 90- 2024.05.22.avi'
fps = 4  # Frames per second
video = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

# Write each image to the video in sorted order
for image in image_files:
    img_path = os.path.join(output_dir, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the video writer object
video.release()

print(f"Video saved as {video_filename}")

Video saved as 032-Hermes - 02244-1 chirp phi 90- 2024.05.22.avi
